# Assignment 3 - Reinforcement Learning

## GridWorlds

This assignment involves finding optimal policies for two grid worlds (CliffWalking and WindyGridWorld) using SARSA and Q learning. Details about WindyGridWorld (Example 6.5) and CliffWalking (Example 6.6) can be found in the following link.
    
    http://incompleteideas.net/book/RLbook2020.pdf


You need gym (version 0.18) and numpy (version 1.20.1) for this assignment. The environment for both problems are provided. 

For Windy Grid World environemnt you also need the file 'WindyGridWorld.py'. 

### Task 1: Learning [5 Marks]

You only need to write the codes for SARSA and Q-learning algorithms. Then do the learning in both 'CliffWalking' and 'Windy Grid World' environments. 

### Task 2: Analysis [5 Marks]   

1. Calculate the average return across the episodes. It gives you a measure of the performance of the algorithm while learning.  

2. Calculate the return after convergence. It gives you a measure of the performance after the learning is completed. 

3. What do you observe from these results?

Install the necessary packages

In [ ]:
!pip install gym==0.22
!pip install numpy==1.20.1
!pip install tqdm 
!pip install pygame

# Task 1: Learning
## Task 1a: Learning in CliffWalking Environment

### Environment for CliffWalking

The board is a 4x12 matrix, with (using NumPy matrix indexing):
    [3, 0] as the start at bottom-left
    [3, 11] as the goal at bottom-right
    [3, 1..10] as the cliff at bottom-center

Each time step incurs -1 reward, and stepping into the cliff incurs -100 reward
and a reset to the start. If an action would take you off the grid, you remain in the previous state.
An episode terminates when the agent reaches the goal.


In [3]:

import gym
import numpy as np
from tqdm import tqdm 

env = gym.make('CliffWalking-v0') # Create the environment #render_mode="human"  human, ansi, 
env.reset() # reset environment to a new, random state
env.render() # Renders the environment for visualization

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T



Here _x_ is the location of the agent, *o* are possible places to go to, *C* is the cliff, and *T* is the target.

In [4]:
num_actions = env.action_space.n 
num_states = env.observation_space.n 

print("Number of actions: ", num_actions)
print("Number of states: ", num_states)

Number of actions:  4
Number of states:  48


In [ ]:
action = 0 # Move up
a = env.step(action) # This is the function we use to interact with the environment
env.render() # Renders the environment for visualization

In [ ]:
# 0 -> UP, 1 -> RIGHT, 2 -> DOWN, 3 -> LEFT
env.reset()
import time
for action in [0, 1, 2, 3]:
    print("Action: ", action)
    time.sleep(1)
    next_state, reward, is_done, info = env.step(action)     # next_state, reward, is_done, info
    print("Next state: ", next_state)
    print("Reward: ", reward)
    print("Done: ",is_done)
    env.render()
env.reset()

As you can see above, each non-terminal action has a reward of -1. 0 -> UP, 1 -> RIGHT, 2 -> DOWN, 3 -> LEFT. The moment the agent falls off the cliff the reward becomes -100 and the agent resets to the start.

In [41]:
# Initialize values 
num_episodes = 500
lr = 100
epsilon = 0.1
alpha = 0.85
gamma = 0.95
discount_factor = 0.9

In [11]:
# Initialize Q function - a simplified version is used here 
# in reality the number of states may be unknown and all states may not be reachable 

# hint: use num_states as the key to a dictionary of lists
Q = np.zeros((num_states, num_actions))

0


In [32]:
def behavioral_policy(state, Q, num_actions, epsilon):
    # Implement the epsilon-greedy policy
    # Don't forget the epsilon-greedy idea
    probs =  np.ones(num_actions,dtype = float) * epsilon / num_actions
    best_action = np.argmax(Q[state])
    probs[best_action] += (1.0 - epsilon)
    
    action = np.argmax(np.random.multinomial(1, probs, size=1)[0])
    return action

In [33]:
# You can use this to check if your algorithm is correct
for i in range(10):
    print(behavioral_policy(0, Q, num_actions, 0.8))

2
0
2
1
2
0
3
3
3
3


### SARSA Learning 

In [58]:


def sarsa(env, Q, num_actions, num_episodes, epsilon, lr):
    # Given to students
    episode_length = [0] * num_episodes
    total_reward_episode = [0] * num_episodes

    for episode in tqdm(range(num_episodes)):
        state = env.reset()
        is_done = False
        # Implement SARSA
        action = behavioral_policy(state, Q, num_actions, epsilon)
        iteration = 0
        while not is_done or iteration < lr:
            state_2, reward, done, _ = env.step(action)
            action_2 = behavioral_policy(state_2, Q, num_actions, epsilon)
            predict = Q[state, action]
            target = reward + gamma * Q[state_2, action_2]
            Q[state, action] = Q[state, action] + alpha * (target - predict)

            episode_length[episode] += 1
            total_reward_episode[episode] += reward
            iteration += 1
            state = state_2
            action = action_2
            is_done = done
    policy = {}
    # Write code here as well
    # Hint: use np.argmax

    return Q, policy, {"rewards": total_reward_episode, "length": episode_length}

In [59]:
# Run SARSA
optimal_sarsa_Q, sarsa_optimal_policy, sarsa_info = sarsa(env, Q, num_actions, num_episodes, epsilon, lr)
print("\nGridWorld SARSA Optimal policy: \n", sarsa_optimal_policy,sarsa_info)

100%|██████████| 500/500 [03:36<00:00,  2.30it/s]


GridWorld SARSA Optimal policy: 
 {} {'rewards': [-2438, -1061, -2634, -3334, -754, -325, -116, -12287, -2040, -630, -392, -2487, -2265, -1630, -10749, -2120, -680, -721, -598, -1166, -16656, -15661, -157, -1192, -2436, -2750, -969, -1074, -5097, -1466, -23156, -12755, -1953, -26413, -1017, -763, -19424, -2934, -1615, -10139, -43735, -4626, -34532, -809, -17252, -972, -2810, -26605, -301, -1875, -141, -3072, -5560, -1042, -46046, -144, -16945, -2192, -886, -11911, -16188, -2452, -261, -10323, -2755, -769, -2843, -11883, -2330, -3765, -3050, -9598, -6546, -17934, -911, -594, -483, -3509, -9522, -8758, -2281, -1438, -8638, -1577, -381, -2338, -1928, -5090, -1253, -1436, -2834, -285, -285, -104, -576, -1061, -641, -1213, -785, -467, -142, -11424, -6908, -3767, -1286, -6386, -3586, -7313, -45902, -41306, -1257, -4819, -32975, -1270, -20188, -73337, -1003, -28011, -256, -8868, -7478, -32715, -3388, -7089, -17823, -13012, -4026, -16414, -3638, -774, -366, -1038, -19003, -27128, -10422, -131

### Q-Learning

In [ ]:
def q_learning(env, Q, num_actions, num_episodes, epsilon, lr):
    # Given to students
    episode_length = [0] * num_episodes
    total_reward_episode = [0] * num_episodes

   
    for episode in tqdm(range(num_episodes)):
        state = env.reset()
        is_done = False
        # Implemnt Q-Learning
        iteration = 0
        while not is_done or iteration < lr:
            action = behavioral_policy(state, Q, num_actions, epsilon)
            next_state, reward, done, _ = env.step(action)
 
            total_reward_episode[episode] += reward
            episode_length[episode] += 1

            # Update Q values
            best_next_action = np.argmax(Q[next_state])  
            td_target = reward + discount_factor * Q[next_state][best_next_action]
            td_delta = td_target - Q[state][action]
            Q[state][action] += alpha * td_delta

            is_done = done
            iteration += 1
            
    policy = {}
    # Write the code here

    return Q, policy, {"rewards": total_reward_episode, "length": episode_length}

In [48]:
# Run Q-Learning 

optimal_Q, q_optimal_policy, q_info = q_learning(env, Q, num_actions, num_episodes, epsilon, lr)
print("\nGridWorld Q-Learning Optimal policy: \n", q_optimal_policy,q_info)

100%|██████████| 500/500 [00:03<00:00, 140.37it/s]


GridWorld Q-Learning Optimal policy: 
 {} {'rewards': [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -199, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -199, -199, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

In [ ]:
# run this cell if you do not have the matplotlib library
# !pip install matplotlib
import matplotlib.pyplot as plt

In [ ]:
def plot_rate(episode_length, total_reward_episode, title):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].plot(episode_length)
    ax[0].set_title("Episode Length over time")
    ax[0].set(xlabel="Episode", ylabel="Length")
    ax[1].plot(total_reward_episode)
    ax[1].set_title("Episode reward over time")
    ax[1].set(xlabel="Episode reward over time", ylabel="Reward")
    fig.suptitle(title)

    plt.show()

In [ ]:
plot_rate(sarsa_info["length"], sarsa_info["rewards"], "GridWorld: SARSA")
plot_rate(q_info["length"], q_info["rewards"], "GridWorld: Q-Learning")

## Task 1b: Learning in Windy Grid world

WindyGridWorld is similar to GridWorld, but with a few differences. You only need to move to the target state. But this time there is a cross-wind across the center of the grid that will push you upwards. In columns 3, 4, 5, and 8 there are winds of strength 1 while in column 6 and 7 there are winds of strength 2. For more details refer Example 6.5 in

 http://incompleteideas.net/book/RLbook2020.pdf

 You only need to change the environment and reuse the SARSA and Q-learning algorithms. 

In [ ]:
#Windy Grid World environment
from WindyGridWorld import WindyGridWorld
env = WindyGridWorld()
env.reset()
env.render()

In [ ]:
num_actions = env.action_space.n 
num_states = env.observation_space.n 

print("Number of actions: ", num_actions)
print("Number of states: ", num_states)

Play around with different learning rates epsilons, and Q initializations to see what is best.

In [ ]:
num_episodes = 1000
lr = 
epsilon = 

In [ ]:
# Initialize Q function - a simplified version is used here 
# in reality the number of states may be unknown and all states may not be reachable 

# hint: use num_states as the key to a dictionary of lists
Q = 

In [ ]:
optimal_sarsa_Q, sarsa_optimal_policy, sarsa_info = sarsa(env, Q, num_actions, num_episodes, epsilon, lr)
print("\n WindyGridWorld SARSA Optimal policy: \n", sarsa_optimal_policy)

In [ ]:
optimal_Q, q_optimal_policy, q_info = q_learning(env, Q, num_actions, num_episodes, epsilon, lr)
print("\n WindyGridWorld Q-Learning Optimal policy: \n", q_optimal_policy)

In [ ]:
plot_rate(sarsa_info["length"], sarsa_info["rewards"], "GridWorld: SARSA")
plot_rate(q_info["length"], q_info["rewards"], "GridWorld: Q-Learning")

# Task 2: Analysis (Comparison of Q-learning and SARSA learning algorithms)

1. Comment on the number of episodes required to converge to the optimal policy for both environments. 
       
2. Discuss the differences in the reward graphs.  

3. Calculate the average return across the episodes for each environment. It gives a measure of the performance of the algorithm while learning (i.e., online performance).  

4. Calculate the return after convergence. It gives you a measure of the performance after the learning is completed (i.e., offline performance). 

5. Briefly summarize your results.
 
 It is advisable to rerun the algorithm a few times to get a clearer understanding of the algorithms.